## Shell Cantilever Example

In [19]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [20]:
import os
import cProfile
os.environ["MKL_THREADING_LAYER"] = "GNU" # for pardiso: if the kernel crashes, try changing "GNU" to "INTEL"

In [21]:
import numpy as np

import dismech

geom = dismech.GeomParams(rod_r0=0,
                          shell_h=1e-3)

material = dismech.Material(density=1200,
                            youngs_rod=0,
                            youngs_shell=2e8,
                            poisson_rod=0,
                            poisson_shell=0.5)

dynamic_3d_sim = dismech.SimParams(static_sim=False,
                                   two_d_sim=False,
                                   use_mid_edge=True,
                                   use_line_search=False,
                                   show_floor=False,
                                   log_data=True,
                                   log_step=1,
                                   dt=1e-2,
                                   max_iter=25,
                                   total_time=1.0,
                                   plot_step=10,
                                   tol=1e-4,
                                   ftol=1e-4,
                                   dtol=1e-2,
                                   sparse=True,
                                   solver='pardiso')

env = dismech.Environment()
env.add_force('gravity', g=np.array([0.0, 0.0, -9.81]))

geo = dismech.Geometry.from_txt(
    '../tests/resources/shell_cantilever/equilateral_mesh_40.txt')

robot = dismech.SoftRobot(geom, material, geo, dynamic_3d_sim, env)

### Time Stepping

We only need to fix the first few nodes

In [22]:
fixed_points = np.array(np.where(robot.state.q[:robot.end_node_dof_index].reshape(-1, 3)[:robot.end_node_dof_index, 0] <= 0.01)[0])
robot = robot.fix_nodes(fixed_points)

stepper = dismech.NewmarkBetaTimeStepper(robot)

In [23]:
cProfile.run('stepper.simulate()', sort='cumtime')

  0%|          | 0/100 [00:00<?, ?it/s]

         1857923 function calls (1854761 primitive calls) in 30.787 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  329/328    0.006    0.000   54.130    0.165 base_events.py:1894(_run_once)
      100    0.085    0.001   30.140    0.301 time_stepper.py:63(step)
      461    0.328    0.001   22.146    0.048 time_stepper.py:160(_compute_forces_and_jacobian)
      461    0.849    0.002   18.956    0.041 triangle_energy.py:97(grad_hess_energy_linear_elastic)
    18018    0.021    0.000   13.627    0.001 einsumfunc.py:1057(einsum)
    18018   13.605    0.001   13.605    0.001 {built-in method numpy._core._multiarray_umath.c_einsum}
      922    0.742    0.001    9.111    0.010 triangle_energy.py:193(compute_dpdp)
      461    0.003    0.000    6.939    0.015 solver.py:35(solve)
      461    0.008    0.000    6.936    0.015 scipy_aliases.py:12(spsolve)
      922    6.498    0.007    6.578    0.007 pardiso_wrapper.py:266(_call

In [24]:
#%lprun -f stepper._TimeStepper__elastic_energies[1].grad_hess_energy_linear_elastic stepper.simulate()